In [1]:
file = "../dataset/data-nottag.set"

In [2]:
import pandas as pd
colnames=['text', 'tag']
def filtered_words(x:tuple):
    #w = .lower()#word_tokenize(x[0],custom_dict=o)
    #ww = [word for word in w if word not in list(thai_stopwords())]
    return (x[0].lower().strip() ,x[1])#(''.join(ww),x[1])

In [3]:
user1 = pd.read_csv(file, names=colnames, header=None,sep="|")

In [4]:
user1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423 entries, 0 to 422
Data columns (total 2 columns):
text    423 non-null object
tag     423 non-null object
dtypes: object(2)
memory usage: 6.7+ KB


In [5]:
from typing import List
from sklearn.pipeline import Pipeline
from pythainlp.ulmfit import process_thai
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [6]:
X = user1.text
y = user1.tag

In [7]:
X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.1, random_state=42)#,shuffle=True)

In [8]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB,BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

In [9]:
f1=[]
report=[]
for model in [MultinomialNB(),
              BernoulliNB(),
              DecisionTreeClassifier(),
              RandomForestClassifier(criterion='entropy'),
              SVC(),
              MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(24, 6), verbose=False, max_iter=2000)
             ]:
    nb = Pipeline([('vect', CountVectorizer(tokenizer=process_thai, ngram_range=(1,2))),
               ('tfidf', TfidfTransformer()),
               ('chi2', SelectKBest(chi2, k='all')),
               ('clf', model),
    ])
    nb.fit(X_train, y_train)
    y_pred = nb.predict(X_test)
    t = accuracy_score(y_pred, y_test)
    del nb
    print(t)
    f1.append(t)
    tt=classification_report(y_test, y_pred)
    #print(tt)
    report.append(tt)

0.8604651162790697
0.46511627906976744


C:\Users\TC\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.8604651162790697


C:\Users\TC\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\TC\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.8837209302325582


C:\Users\TC\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\TC\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.13953488372093023


C:\Users\TC\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.9767441860465116


In [10]:
print(report[-1])

              precision    recall  f1-score   support

       alert       1.00      1.00      1.00         6
         fan       1.00      1.00      1.00         3
       light       1.00      1.00      1.00         5
       music       1.00      1.00      1.00        10
        news       1.00      0.75      0.86         4
    religion       1.00      1.00      1.00         4
         sos       0.83      1.00      0.91         5
     weather       1.00      1.00      1.00         6

    accuracy                           0.98        43
   macro avg       0.98      0.97      0.97        43
weighted avg       0.98      0.98      0.98        43

